In [2]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime

In [25]:
plikembd = open("../embeddings_small.txt")

embeddingscuda_chars = {}
embeddingscuda_chars['🈳'] = 0

embeddingscuda = torch.tensor([]).to("cuda:0")
embeddingscuda =  torch.cat((embeddingscuda, torch.zeros((400)).to("cuda:0")), 0)

#print(embeddingscuda.shape)

for cnt, line in enumerate(plikembd.readlines()):
    exp1 = line.replace("\n", "").split(":")
    
    znak = exp1[0]
    embedding = torch.tensor([float(liczba) for liczba in exp1[1].replace(" ", "").replace("[", "").replace("]", "").split(",\t")[:-1]]).to("cuda:0")
    
    embeddingscuda_chars[znak] = cnt+1
    embeddingscuda = torch.cat((embeddingscuda, embedding), 0)

embeddingscuda = embeddingscuda.reshape(82, -1)

print(embeddingscuda.shape)
#print(embeddingscuda[embeddingscuda_chars['a']])

torch.Size([82, 400])


In [26]:
embeddingscuda

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.0851,  1.4174, -0.8968,  ..., -0.5574,  1.2953, -0.1756],
        [ 0.9934, -0.4756, -0.5951,  ...,  0.8698,  0.2920,  0.0168],
        ...,
        [-0.2883,  0.0300, -0.3970,  ...,  0.0964, -0.2076, -0.5465],
        [ 0.2481, -0.3300, -0.0937,  ...,  0.3352, -0.1689,  0.2105],
        [ 0.2100, -0.0179, -0.3131,  ...,  0.0312, -0.2439,  0.2052]],
       device='cuda:0')

In [72]:
class PositionalEmbeddings(nn.Module):
    def __init__(self, dim_1, dim_2):
        super(PositionalEmbeddings, self).__init__()
        
        self.positional_encs = nn.Parameter(torch.zeros(dim_1, dim_2))
#         print(self.w.device)


        for pos in range(dim_1):
            for i in range(0, dim_2, 2):
                self.positional_encs[pos, i] = math.sin(pos / (10000 ** ((i) / dim_2)))
                self.positional_encs[pos, i + 1] = math.cos(pos / (10000 ** ((i + 1) / dim_2)))
    
        
        self.dim_1 = dim_1
        self.dim_2 = dim_2
        
        
    def forward(self, embd):
        return embd+self.positional_encs

In [75]:
pe = PositionalEmbeddings(82, 400).cuda()

In [76]:
xx = pe(embeddingscuda)

In [78]:
zz = xx.cpu().detach().numpy()

In [79]:
plt.figure(figsize = (5,5))
plt.imshow(zz, aspect='auto')

FigureCanvasNbAgg()

![](https://miro.medium.com/max/380/1*2vyKzFlzIHfSmOU_lnQE4A.png)

In [ ]:
def Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        
        self.MH = nn.MultiheadAttention